In [44]:
from bg_parser import extract_product, save_csv

In [55]:
url = "https://www.biblio-globus.ru/product/11061311"
row = extract_product(url)

if row is None:
    print("❌ Пропущено (не книга):", url)
else:
    print("Parsed:", {k: (str(v)[:120] + ("…" if isinstance(v, str) and len(v) > 120 else "")) for k, v in row.items()})
    save_csv(row, append=True)
    print("✅ Saved to books.csv")


Parsed: {'page_url': 'https://www.biblio-globus.ru/product/11061311', 'image_url': 'https://static1.bgshop.ru/imagehandler.ashx?filename=11061311.jpg&width=550', 'author': 'Лупенко П.А.', 'title': 'Депрессия: выход есть. Возможно, самая радостная книга в твоей жизни', 'annotation': 'Только тот, кто прошёл через ад депрессии, по-настоящему понимает, как выбраться из него. Пётр Лупенко — коуч по самораз…'}
✅ Saved to books.csv


In [ ]:
# collect_from_catalog_new.py
# Требует: requests, beautifulsoup4, lxml, pandas
import time
import re
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

# --- хедера как у браузера ---
HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/120.0.0.0 Safari/537.36"),
    "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8",
    "Referer": "https://www.biblio-globus.ru/",
}

# быстрая загрузка страницы каталога
def get_soup(url: str) -> BeautifulSoup:
    r = requests.get(url, headers=HEADERS, timeout=20)
    r.raise_for_status()
    return BeautifulSoup(r.text, "lxml")

# меняем только параметр ?page= в URL каталога
def set_page_param(category_url: str, page_num: int) -> str:
    parts = urlparse(category_url)
    q = parse_qs(parts.query)
    q["page"] = [str(page_num)]
    new_query = urlencode(q, doseq=True)
    return urlunparse(parts._replace(query=new_query))

# достаём все product-id с одной страницы каталога
def extract_product_ids_from_catalog_soup(soup: BeautifulSoup) -> list[int]:
    ids = set()
    # 1) быстрый и надёжный вариант: ищем все ссылки на /product/<id>
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if "/product/" in href:
            pid = href.split("/product/")[1].split("?")[0].strip("/")
            if pid.isdigit():
                ids.add(int(pid))
    # 2) иногда ссылки лежат в data-атрибутах — можно продублировать regex по всему HTML
    if not ids:
        html = soup.decode()
        for pid in re.findall(r'/product/(\d+)', html):
            ids.add(int(pid))
    return sorted(ids)

# основной сборщик: идём по page=1..N, пока приходят новые id
def collect_product_urls_from_catalog(category_url: str,
                                      start_page: int = 1,
                                      max_pages: int = 1000,
                                      sleep: float = 0.4,
                                      empty_streak_break: int = 2) -> list[str]:
    seen, urls = set(), []
    empty_streak = 0
    for page in range(start_page, start_page + max_pages):
        page_url = set_page_param(category_url, page)
        try:
            soup = get_soup(page_url)
        except Exception as e:
            print(f"[page={page}] ❌ ошибка загрузки: {e}")
            break

        page_ids = extract_product_ids_from_catalog_soup(soup)
        new_ids = [pid for pid in page_ids if pid not in seen]

        if not new_ids:
            empty_streak += 1
            print(f"[page={page}] 0 новых ID (подряд {empty_streak})")
            if empty_streak >= empty_streak_break:
                break
        else:
            empty_streak = 0
            for pid in new_ids:
                seen.add(pid)
                urls.append(f"https://www.biblio-globus.ru/product/{pid}")
            print(f"[page={page}] +{len(new_ids)} новых (всего {len(urls)})")

        time.sleep(sleep)

    return urls

# ---- подключаем твой модуль парсинга карточек ----
from bg_parser import extract_product, save_csv  # уже сделали ранее

def run_catalog_to_csv(category_url: str,
                       csv_path: str = "books.csv",
                       append: bool = False,
                       max_pages: int = 1000,
                       target_rows: int | None = None):
    product_urls = collect_product_urls_from_catalog(
        category_url, start_page=1, max_pages=max_pages, sleep=0.2
    )
    print(f"🥳 Собрано ссылок: {len(product_urls)}. Начинаю парсинг карточек…")

    rows, kept = [], 0
    for i, url in enumerate(product_urls, 1):
        try:
            row = extract_product(url)  # вернёт dict или None (если не книга/аудио)
            if row is None:
                if i % 50 == 0:
                    print(f"{i:04d}. ⏭️ пропущено (не книга/аудио) {url}")
                continue
            rows.append(row)
            kept += 1
            # периодически сохраняем, чтобы не потерять прогресс
            if len(rows) >= 50:
                save_csv(rows, path=csv_path, append=append)
                append = True
                rows.clear()
                print(f"💾 промежуточно сохранено, всего книг: {kept}")
            if target_rows and kept >= target_rows:
                break
        except Exception as e:
            print(f"{i:04d}. ❌ ошибка парсинга {url}: {e}")

        time.sleep(0.2)  # вежливая задержка

    if rows:
        save_csv(rows, path=csv_path, append=append)

    print(f"🎉 Готово: сохранено книг {kept} → {csv_path}")

# ======== пример запуска ========
# if __name__ == "__main__":
#     CATEGORY = "https://www.biblio-globus.ru/catalog/category?id=226&page=1&sort=0&instock=&isdiscount="
#     run_catalog_to_csv(CATEGORY, csv_path="books.csv", append=False, max_pages=1000, target_rows=5000)


https://www.biblio-globus.ru/product/11061311

In [54]:
CATEGORY = "https://www.biblio-globus.ru/catalog/category?id=226&page=1&sort=0&instock=&isdiscount="
run_catalog_to_csv(CATEGORY, csv_path="books.csv", append=False, max_pages=1000, target_rows=5000)

[page=1] +12 новых (всего 12)
[page=2] +12 новых (всего 24)
[page=3] +12 новых (всего 36)
[page=4] +12 новых (всего 48)
[page=5] +12 новых (всего 60)
[page=6] +12 новых (всего 72)
[page=7] +12 новых (всего 84)
[page=8] +12 новых (всего 96)
[page=9] +12 новых (всего 108)
[page=10] +12 новых (всего 120)
[page=11] +12 новых (всего 132)
[page=12] +12 новых (всего 144)
[page=13] +12 новых (всего 156)
[page=14] +12 новых (всего 168)
[page=15] +12 новых (всего 180)
[page=16] +12 новых (всего 192)
[page=17] +12 новых (всего 204)
[page=18] +12 новых (всего 216)
[page=19] +12 новых (всего 228)
[page=20] +12 новых (всего 240)
[page=21] +12 новых (всего 252)
[page=22] +12 новых (всего 264)
[page=23] +12 новых (всего 276)
[page=24] +12 новых (всего 288)
[page=25] +12 новых (всего 300)
[page=26] +12 новых (всего 312)
[page=27] +12 новых (всего 324)
[page=28] +12 новых (всего 336)
[page=29] +12 новых (всего 348)
[page=30] +12 новых (всего 360)
[page=31] +12 новых (всего 372)
[page=32] +12 новых (всег